In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

ImportError: Unable to import required dependencies:
pytz: No module named 'pytz'

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Input, Dropout
from sklearn.metrics import confusion_matrix
from tensorflow.keras import layers
import joblib
from IPython.display import clear_output
# from google.colab import files

In [ ]:
# from google.colab import files
# files.upload()
# clear_output(wait=True)


In [ ]:
# !mkdir -p ~/.kaggle
# !mv kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d mlg-ulb/creditcardfraud
!unzip -o creditcardfraud.zip
!rm -rf creditcardfraud.zip

Dataset URL: https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud
License(s): DbCL-1.0
 59%|███████████████████████                | 39.0M/66.0M [00:00<00:00, 214MB/s]
100%|███████████████████████████████████████| 66.0M/66.0M [00:00<00:00, 228MB/s]
Archive:  creditcardfraud.zip
  inflating: creditcard.csv          


In [ ]:
df = pd.read_csv('/kaggle/input/creditcardfraud/creditcard.csv')

X = df.drop(columns=['Class'])
y = df['Class']



In [ ]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1, shuffle=True)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_test, X_val , y_test, y_val = train_test_split(X_test, y_test, train_size=0.5, random_state=1, shuffle=True)

NameError: name 'train_test_split' is not defined

In [ ]:
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [ ]:
y_train = y_train.values
y_test = y_test.values
y_val = y_val.values

In [ ]:
# TPU configuration
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
    print("Running on TPU")
except ValueError:
    print("TPU not found, using CPU/GPU instead")
    strategy = tf.distribute.get_strategy()

TPU not found, using CPU/GPU instead


In [ ]:
with strategy.scope():
    model = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(128, activation='relu'),
        Dropout(0.6),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',  # Keep the loss suitable for binary classification
        metrics=[
            tf.keras.metrics.Recall(),  # Prioritize recall
            tf.keras.metrics.Precision(),  # Optionally track precision as well
            'accuracy'  # Still track accuracy for reference
        ]
    )


early_stopping = EarlyStopping(monitor='val_recall', patience=10, mode='max', restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)

# Class weights to priotrize class 1
class_weight = {0: 1., 1: 16}

In [ ]:
batch_size = 32
history = model.fit(X_train, y_train,
                    epochs=20,
                    batch_size=batch_size,
                    validation_data=(X_val, y_val),
                    verbose=1,
                    callbacks=[early_stopping, reduce_lr],
                    class_weight=class_weight)


Epoch 1/20
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.9932 - loss: 0.1078 - precision_5: 0.3323 - recall_5: 0.6262 - val_accuracy: 0.9990 - val_loss: 0.0061 - val_precision_5: 0.7021 - val_recall_5: 0.7021 - learning_rate: 0.0010
Epoch 2/20
  42/7121 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.9996 - loss: 0.0072 - precision_5: 0.0000e+00 - recall_5: 0.0000e+00

/opt/conda/lib/python3.10/site-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_recall` which is not available. Available metrics are: accuracy,loss,precision_5,recall_5,val_accuracy,val_loss,val_precision_5,val_recall_5
  current = self.get_monitor_value(logs)


7121/7121 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - accuracy: 0.9990 - loss: 0.0341 - precision_5: 0.6572 - recall_5: 0.8306 - val_accuracy: 0.9991 - val_loss: 0.0047 - val_precision_5: 0.7391 - val_recall_5: 0.7234 - learning_rate: 0.0010
Epoch 3/20
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - accuracy: 0.9990 - loss: 0.0405 - precision_5: 0.6852 - recall_5: 0.8122 - val_accuracy: 0.9990 - val_loss: 0.0057 - val_precision_5: 0.6939 - val_recall_5: 0.7234 - learning_rate: 0.0010
Epoch 4/20
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - accuracy: 0.9987 - loss: 0.0272 - precision_5: 0.5870 - recall_5: 0.8452 - val_accuracy: 0.9989 - val_loss: 0.0109 - val_precision_5: 0.6230 - val_recall_5: 0.8085 - learning_rate: 0.0010
Epoch 5/20
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.9989 - loss: 0.0304 - precision_5: 0.6423 - recall_5: 0.8242 - val_accuracy: 0.9991 - val_loss: 0.0051 - val_precision_5: 0.7234 - val_recall_5: 0.7234 - learning_rate: 0.0010
Epoch 6/20
7121/7121 ━━━━━━━━━━

In [ ]:
results = model.evaluate(X_test, y_test, verbose=1)

891/891 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9995 - loss: 0.0041 - precision_5: 0.8863 - recall_5: 0.8282


In [ ]:
def print_scores(y_true, y_pred, model_name):
    print(f"{model_name} Precision: {precision_score(y_true, y_pred):0.2f}")
    print(f"{model_name} Recall: {recall_score(y_true, y_pred):0.2f}")
    print(f"{model_name} F1-Score: {f1_score(y_true, y_pred):0.2f}")

In [ ]:
y_train_pred_ = model.predict(X_train)

7121/7121 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step


In [ ]:
y_val_pred_ = model.predict(X_val)

891/891 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


In [ ]:
thresholds = [0.3,0.4,0.48,0.5 ,0.55,0.56,0.59, 0.6,0.7]
for threshold in thresholds:
    y_val_pred = (y_val_pred_ > threshold).astype(int).flatten()
    print(f'threshold: {threshold}')
    print_scores(y_val, y_val_pred, 'Neural Network (Validation)')
threshold = 0.55

threshold: 0.3
Neural Network (Validation) Precision: 0.64
Neural Network (Validation) Recall: 0.81
Neural Network (Validation) F1-Score: 0.72
threshold: 0.4
Neural Network (Validation) Precision: 0.68
Neural Network (Validation) Recall: 0.81
Neural Network (Validation) F1-Score: 0.74
threshold: 0.48
Neural Network (Validation) Precision: 0.77
Neural Network (Validation) Recall: 0.77
Neural Network (Validation) F1-Score: 0.77
threshold: 0.5
Neural Network (Validation) Precision: 0.77
Neural Network (Validation) Recall: 0.77
Neural Network (Validation) F1-Score: 0.77
threshold: 0.55
Neural Network (Validation) Precision: 0.77
Neural Network (Validation) Recall: 0.77
Neural Network (Validation) F1-Score: 0.77
threshold: 0.56
Neural Network (Validation) Precision: 0.77
Neural Network (Validation) Recall: 0.77
Neural Network (Validation) F1-Score: 0.77
threshold: 0.59
Neural Network (Validation) Precision: 0.77
Neural Network (Validation) Recall: 0.77
Neural Network (Validation) F1-Score: 

In [ ]:
y_train_pred = (y_train_pred_ > threshold).astype(int).flatten()
print(f'threshold: {threshold}')
print_scores(y_train, y_train_pred, 'Neural Network (Train)')

threshold: 0.55
Neural Network (Train) Precision: 0.86
Neural Network (Train) Recall: 0.85
Neural Network (Train) F1-Score: 0.86


In [ ]:
y_pred_ = (model.predict(X_test))

891/891 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


In [ ]:
print(f'threshold: {threshold}')
y_pred = (y_pred_ > threshold).astype("int32")
print_scores(y_test, y_pred, 'Neural Network (Test)')

threshold: 0.55
Neural Network (Test) Precision: 0.92
Neural Network (Test) Recall: 0.82
Neural Network (Test) F1-Score: 0.87


In [ ]:
model.save('neural_network_model_pca_80.keras')

In [ ]:
import tensorflow as tf
import keras

print(f'TensorFlow version: {tf.__version__}')
print(f'Keras version: {keras.__version__}')

TensorFlow version: 2.16.1
Keras version: 3.3.3
